In [1]:
import os
import glob
import pickle
import nibabel as nib
import numpy as np
from itkwidgets import view
from skimage.segmentation import flood_fill
from skimage.morphology import skeletonize_3d, binary_opening, binary_closing, binary_dilation, ball, cube

import vtk
from vtk.util.numpy_support import vtk_to_numpy, numpy_to_vtk
import itertools
import numpy as np
import networkx as nx
from scipy import ndimage
from skimage import measure
from functools import reduce
from heapq import heappush, heappop
from skimage.segmentation import flood_fill
from skimage.morphology import skeletonize_3d, binary_opening, binary_closing, binary_dilation, ball, cube

from scipy.sparse.csgraph import shortest_path
from sklearn.neighbors import radius_neighbors_graph
from scipy.sparse.csgraph import connected_components
from anytree import Node

import torch
from torch import nn, Tensor
import torch.nn.functional as F

def np_to_polydata(np_mat):
    polyData = vtk.vtkPolyData()
    numberOfPoints = len(np_mat)
    points = vtk.vtkPoints()
    for x, y, z in np_mat:
        points.InsertNextPoint(x, y, z)
    lines = vtk.vtkCellArray()
    lines.InsertNextCell(numberOfPoints)
    for i in range(numberOfPoints):
        lines.InsertCellPoint(i)
    polyData.SetPoints(points)
    polyData.SetLines(lines)
    return polyData

def np_to_points(np_mat):
    pts = vtk.vtkPoints()
    pts.SetData(numpy_to_vtk(np_mat))
    return pts

def interp(curve, sample_spacing):
    curve = np.array(curve)
    polyData = np_to_polydata(curve)
    spline = vtk.vtkSplineFilter()
    cardinal_spine = vtk.vtkCardinalSpline()
    spline.SetInputDataObject(polyData)
    spline.SetSpline(cardinal_spine)
    spline.SetSubdivideToLength()
    spline.SetLength(sample_spacing)
    spline.Update()
    equal_length_pts = vtk_to_numpy(spline.GetOutput().GetPoints().GetData())
    return equal_length_pts

def interp_load_txt_from_dir(dirname, prefix=''):
    txtfiles = glob.glob(os.path.join(dirname, '%s*.txt'%(prefix)))
    points = []
    poly_points = []
    for i in range(0, len(txtfiles)):
        data = np.loadtxt(txtfiles[i])
        if len(data.shape) <= 1:
            continue
        poly_data = interp(data, sample_spacing=0.35)
        points.append(data)
        poly_points.append(poly_data)
    return points, poly_points


mask_nii = nib.load('/mnt/users/drwise_runtime_env_nightly/data1/inputdata/1004812/058AD1E9/259A3499_CTA/mask_source/mask_vessel.nii.gz')#058AD1E9/259A3499_CTA/mask_source/
mask_raw = mask_nii.get_fdata()
mask_raw = (mask_raw > 0).astype(int)

pts, poly_pts = interp_load_txt_from_dir('/mnt/users/drwise_runtime_env_nightly/data1/inputdata/1004812/058AD1E9/259A3499_CTA/curve_source')
pts = np.concatenate(pts, axis=0)
poly_pts = np.concatenate(poly_pts, axis=0)

mask_pts = np.stack(np.where(mask_raw>0), axis=1)
point_sets = [mask_pts]

point_sets.append(pts)
point_sets.append(poly_pts)
    
view(point_sets=point_sets)

/mnt/users/.local/lib/python3.7/site-packages/dicom/__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)


ImportError: libGL.so.1: cannot open shared object file: No such file or directory